# Imports & reading the data

In [ ]:
import json
import torch
import random
from tqdm import tqdm
from prettytable import PrettyTable
import spacy
from spacy.training import Example
!python -m spacy download ru_core_news_lg

import warnings
warnings.filterwarnings("ignore")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 513.4/513.4 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.2/53.2 kB 761.6 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 23.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
with open("../train.jsonl") as json_file:
    json_list = list(json_file)

# sentences and ners from `train.jsonl`
train_sent = list()
train_ners = list()

for json_str in json_list:
    result = json.loads(json_str)
    train_sent.append(result["sentences"])

    train_ners.append(
        sorted(
            sorted([(result["sentences"][start:end+1], start, end, t) for start, end, t in result["ners"]], key=lambda l:l[2], reverse=True),
            key=lambda l: l[1]
        )
    )

with open("../test.jsonl") as json_file:
    json_list = list(json_file)

# sentences and corresponding ids from `test.jsonl`
test_sent = list()
test_idx = list()

for json_str in json_list:
    result = json.loads(json_str)
    test_sent.append(result["senences"]) # sEnENceS
    test_idx.append(result["id"])

# Model code

In [ ]:
def get_entities_spacy(sent, recursion_depth=6):
    """
    This function takes a sentence and a maximum recursion depth
    and returns a list of all found entities without duplicates.

    Arguments:
        sent (str): a string where we need to find the entities
        recursion_depth (int): maximum recursion depth we may call

    Returns:
        List(Tuple(str, int, int, str)): a list of all found entities
                                         in a format (<word>, <start idx>, <end idx inclusive>, <label>)
    """
    # stores entities from current sentence
    res = list()
    doc = nlp(sent)

    # stores entities found from sub-calls
    sub_ents = list()

    for ent in doc.ents:
        # gets word boundaries
        a, b = ent.start_char, ent.end_char
        # stores the result in res
        res.append((doc.text[a:b], a, b-1, ent.label_))

        if recursion_depth and res[-1][0] != sent and len(res[-1][0].split())>1:
            # makes a recursive call for each found entity
            sub_ent = get_entities_spacy(res[-1][0], recursion_depth-1)

            # writes the found entity into sub_ents
            sub_ent = [(x[0], x[1]+a, x[2]+a, x[3]) for x in sub_ent]
            sub_ents += sub_ent

    # returns a list of all unique entities in this call and all sub-calls
    return list(set(res + sub_ents))

# Dataset splitting

In [ ]:
def fun(text, ners):
    """
    Unfolds all nested entities recursively.

    For example, an input:

    "Ilya works in Moscow University" -> ["Ilya", "Person"], ["Moscow University", "Organization"], ["Moscow", "City"]

    would be split into two separate training examples:

    "Ilya works in Moscow University" -> ["Ilya", "Person"], ["Moscow University", "Organization"]
    "Moscow University" -> ["Moscow", "City"]

    Parameters:
        text (str): a full sentence to be split
        ners (List(Tuple(str, int, int, str))): a list of entities to be split

    Returns:
        List(Tuple(str, List(Tuple(str, int, int, str)))): a list of unfolded entities in a format (<sentence>, <list of ners>)
    """
    # sorts the entities by end idx and start idx
    ners.sort(reverse=True, key=lambda l:l[2])
    ners.sort(key=lambda l:l[1])

    res = []

    # shows whether this entity is nested
    included = [False] * len(ners)
    # stores parent text and location for nested values
    par_sent_loc = text
    shift_loc = 0

    # a dict that stores all nested entities and their parent text
    to_process = dict()

    for i, item in enumerate(ners):
        # if entity is not nested, then store its text and position
        if not res or item[1] > res[-1][2]:
            res.append(item)
            included[i] = True
            par_sent_loc = item[0]
            shift_loc = item[1]

        # if entity is nested, stores it in `to_process`
        if not included[i]:
            l = to_process.get(par_sent_loc, list())
            l.append((item[0], item[1]-shift_loc, item[2]-shift_loc, item[3]))
            l = list(set(l))
            to_process[par_sent_loc] = l

    answer = [[text, res]]

    # recursively calls this function for each nested entity
    for k, v in to_process.items():
        answer += fun(k, v)

    return answer

# Training

In [ ]:
# 30 sentences will be used for eval, the rest for training
X_eval = train_sent[:30]
y_eval = train_ners[:30]
X_train = train_sent[30:]
y_train = train_ners[30:]

# transforming the training set with the function above
training_set = list()
for sent, ner in zip(X_train, y_train):
    training_set += fun(sent, ner)

In [ ]:
nlp = spacy.load("ru_core_news_lg")

# disabling all pipes except for `ner`
disabled_pipes = list()
for pipe_name in nlp.pipe_names:
    if pipe_name != "ner":
        nlp.disable_pipes(pipe_name)
        disabled_pipes.append(pipe_name)

optimizer = nlp.create_optimizer()

n_epochs = 20
batch = 16
for i in range(n_epochs):
    losses = {}
    pbar = tqdm(total=len(training_set)//batch+1, position=0, leave=True)
    pbar.set_description_str(f"Epoch [{i+1}/{n_epochs}]")

    for j in range(len(training_set)//batch+1):
        # creating a list of randomly selected examples from training set
        example = list()
        for item in random.sample(training_set, batch):
            entities = [(x[1], x[2]+1, x[3]) for x in item[1]]
            doc = nlp.make_doc(item[0])
            example.append(Example.from_dict(doc, {"entities": entities}))
        # training nlp ner on these examples
        try: nlp.update(example, drop=0.3, losses=losses, sgd=optimizer)
        except: pass
        pbar.update(1)
        pbar.set_postfix_str(f"Loss: {round(losses['ner']/(j+1), 2)}")

# enabling the pipes back
for pipe_name in disabled_pipes:
    nlp.enable_pipe(pipe_name)

Epoch [20/20]: 100%|██████████| 303/303 [01:00<00:00,  5.01it/s, Loss: 41.83]

# Finy-tuning

In [ ]:
def eval_f1(pred, true):
    """
    Evaluation function for NER. Calculates the f1-score like this:
    true positives are the entities correctly classified
    false positives are predicted entities that aren't in ground truth
    false negatives are true entities that weren't predicted.
    The f1-score is then calculated as usual

    Format of input variables:
    List of tuples (<word>, <start char idx>, <end char idx (inclusive)>, <type>)
    """
    true = set(true)
    pred = set(pred)

    true_positive = len(true & pred)
    false_positive = len(pred - true)
    false_negative = len(true - pred)

    if not true_positive:
        return 0

    recall = true_positive / (true_positive+false_positive)
    precision = true_positive / (true_positive+false_negative)

    return 2*precision*recall / (precision+recall)

In [ ]:
items_to_test = 30
max_depth = 7

table = PrettyTable()
table.field_names = [""] + [f"depth={x}" for x in range(max_depth)]

pbar = tqdm(total=max_depth*items_to_test, position=0)

row = ["F1-score"]

for depth in range(max_depth):
    total_f1 = 0

    for idx in range(items_to_test):
        pbar.set_postfix_str(f"Depth={depth}, item={idx+1}, f1-score: {round(100*total_f1/(idx+1), 3)}%")

        pred = get_entities_spacy(train_sent[idx], recursion_depth=depth)
        true = train_ners[idx]

        f1_score = eval_f1(pred, true)
        total_f1 += f1_score

        pbar.update(1)

    row.append(f"{round(100*total_f1/items_to_test, 2)}%")
table.add_row(row)

print()
print(table)

100%|██████████| 210/210 [01:23<00:00,  2.86it/s, Depth=6, item=30, f1-score: 68.58%]


+----------+---------+---------+---------+---------+---------+---------+---------+
|          | depth=0 | depth=1 | depth=2 | depth=3 | depth=4 | depth=5 | depth=6 |
+----------+---------+---------+---------+---------+---------+---------+---------+
| F1-score |  66.36% |  70.73% |  71.14% |  71.21% |  71.19% |  71.19% |  71.19% |
+----------+---------+---------+---------+---------+---------+---------+---------+


# Predictions

In [ ]:
pbar = tqdm(
    total=len(test_sent), position=0
)

depth = 5

with open("test.jsonl", "w") as f:
    for s, idx in zip(test_sent, test_idx):
        ners = get_entities_spacy(s, depth)
        ners = [ [x[1], x[2], x[3]] for x in ners]
        ners = {"ners": ners, "id": idx}
        ners = json.dumps(ners)
        f.write(f"{ners}\n")
        pbar.update(1)

print()
!zip test test.jsonl

100%|██████████| 65/65 [00:31<00:00,  1.04it/s]


updating: test.jsonl (deflated 76%)
